In [ ]:
import os
import numpy
from plotly import offline as plotly
from plotly import graph_objects
from plotly.subplots import make_subplots
from scipy import stats
from statsmodels.stats import multitest

import capblood_seq
from capblood_seq import config

In [ ]:
# Load the dataset. This downloads it if it doesn't exist already, and loads it into memory
dataset = capblood_seq.load_dataset(data_directory="data", pipeline_name="visualization")

In [ ]:
# For this plot, we're going to load all the leaf nodes of the cell type/subtype heirarchy
CELL_TYPES = [
    "CD4 T Cells",
    "CD8 T Cells",
    "NK Cells",
    "B Cells",
    "CD14 Monocytes",
    "CD16 Monocytes"
]

# A list of marker genes to inspect
MARKER_GENE_LIST = [
    "CD3D",
    "IL7R",
    "CD8A",
    "CD8B",
    "NKG7",
    "GNLY",
    "MS4A1",
    "CD79A",
    "CD19",
    "LYZ",
    "CD14",
    "FCGR3A",
    "MS4A7"
]

# For each marker gene, which cell types it should be highlighted in
MARKER_GENE_LIST_CELL_TYPES = [
    ["CD4 T Cells", "CD8 T Cells", "NK Cells"],
    ["CD4 T Cells"],
    ["CD8 T Cells"],
    ["CD8 T Cells"],
    ["CD8 T Cells", "NK Cells"],
    ["NK Cells"],
    ["B Cells"],
    ["B Cells"],
    ["CD14 Monocytes", "CD16 Monocytes"],
    ["CD14 Monocytes"],
    ["CD14 Monocytes"],
    ["CD16 Monocytes"],
    ["CD16 Monocytes"]
]

In [ ]:
figure = make_subplots(
    rows=len(CELL_TYPES),
    cols=len(MARKER_GENE_LIST),
    shared_xaxes=True,
    shared_yaxes=False,
    vertical_spacing=0,
    horizontal_spacing=-0.02
)

cell_type_gene_counts = {}

for cell_type_index, cell_type in enumerate(CELL_TYPES):
        
    for gene_index, gene in enumerate(MARKER_GENE_LIST):
    
        x_values = []
        y_values = []
            
        gene_counts = dataset.get_transcript_counts(
            cell_type=cell_type,
            genes=gene,
            normalized=True
        ).reshape((-1,))

        x_values.extend([gene] * len(gene_counts))
        y_values.extend(gene_counts)
            
        if cell_type in MARKER_GENE_LIST_CELL_TYPES[gene_index]:
            color = config.CELL_TYPE_HIERARCHICAL_COLORS[cell_type]
        else:
            color = "grey"
    
        violin_trace = graph_objects.Violin(
            x=x_values,
            y=y_values,
            line={
                "width": 0.75
            },
            marker={
                "size": 1,
                "color": color
            },
            points=False,
            hoverinfo="none",
            bandwidth=0.065
        )

        figure.add_trace(violin_trace, row=cell_type_index + 1, col=gene_index + 1)
    
        figure.update_yaxes(
            {
                "visible": False,
            },
            row=cell_type_index + 1,
            col=gene_index + 1
        )
        figure.update_xaxes(
            {
                "tickangle": -90
            },
            row=cell_type_index + 1,
            col=gene_index
        )
        
    figure.update_yaxes(
        {
            "visible": True,
            "side": "right",
            "tickvals": [0, 2],
            "range": [-0.5, 2.75]
        },
        row=cell_type_index + 1,
        col=len(MARKER_GENE_LIST)
    )

layout = {
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "showlegend": False,
    "violingroupgap": 0,
    "annotations": [
        graph_objects.layout.Annotation(
            x=-0.4,
            y=1 - (cell_type_index/len(CELL_TYPES) + 0.75/len(CELL_TYPES)),
            xref="x",
            yref="paper",
            xanchor="right",
            text=cell_type.replace("_", " "),
            showarrow=False,
            align="right"
        )
        for cell_type_index, cell_type in enumerate(CELL_TYPES)
    ],
    "height": 700,
    "width": 700,
    "margin": graph_objects.layout.Margin(
        l=100,
        r=25
    )
}

figure.update_layout(layout)

plotly.iplot(figure)

In [ ]:
figure.write_image(os.path.join("figures", "cell_type_marker_gene_distributions.svg"))
figure.write_html(os.path.join("figures", "cell_type_marker_gene_distributions.html"))